In [20]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.data_manager import SHREDDataManager
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os

# Initialize SHREDDataManager
manager = SHREDDataManager(
    lags = 20,
    train_size = 0.7,
    val_size = 0.15,
    test_size = 0.15,
    scaling = "minmax",
    compression = 20,
    reconstructor=True,
    time=np.arange(0, 2000),
    forecastor=True)


# Add data to manager (no sensors)
input_folder = "data4fields_npz"
for filename in os.listdir(input_folder):
    if filename.endswith(".npz") and filename != 'ni_3D.npz':
        file_path = os.path.join(input_folder, filename)
        manager.add(
            data=file_path,
        )
        print(f"Processed: {filename}")


# Add data to manager (with sensors)
ni_3D = np.load('data4fields_npz/ni_3D.npz')
ni_3D = ni_3D[ni_3D.files[0]]
mobile_sensors = [
    perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=True),
    perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=False)
]
manager.add(
    data=ni_3D,
    random_sensors=2,
    stationary_sensors=[(7, 9), (0,0)],
    mobile_sensors=mobile_sensors
)


self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)
Processed: Jex_3D.npz
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)
Processed: Jey_3D.npz
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 20)
Processed: Jez_3D.npz
self.full_state_data flattened (2000, 65792)
n_components 20
V.shape (20, 65792)
compressed full_state_data: (2000, 20)
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transforme

In [22]:
# Get train/valid/test datasets
train_set, valid_set, test_set = manager.preprocess()

# Print dataset shapes
print('Data Shapes:')
print ('Reconstructor Data')
print('train X:', train_set.reconstructor.X.shape)
print('train Y:', train_set.reconstructor.Y.shape)
print('valid X:', valid_set.reconstructor.X.shape)
print('valid Y:', valid_set.reconstructor.Y.shape)
print('test X:', test_set.reconstructor.X.shape)
print('test Y:', test_set.reconstructor.Y.shape)


print ('Forecaster Data')
print('train X:', train_set.forecaster.X.shape)
print('train Y:', train_set.forecaster.Y.shape)
print('valid X:', valid_set.forecaster.X.shape)
print('valid Y:', valid_set.forecaster.Y.shape)
print('test X:', test_set.forecaster.X.shape)
print('test Y:', test_set.forecaster.Y.shape)

Data Shapes:
Reconstructor Data
train X: torch.Size([1400, 21, 6])
train Y: torch.Size([1400, 80])
valid X: torch.Size([300, 21, 6])
valid Y: torch.Size([300, 80])
test X: torch.Size([300, 21, 6])
test Y: torch.Size([300, 80])
Forecaster Data
train X: torch.Size([1399, 21, 6])
train Y: torch.Size([1399, 6])
valid X: torch.Size([299, 21, 6])
valid Y: torch.Size([299, 6])
test X: torch.Size([299, 21, 6])
test Y: torch.Size([299, 6])


In [23]:
# Verfiy forecaster data
print("Verify forecaster train set...")
print(torch.all(train_set.forecaster.X[1:, -1, :] == train_set.forecaster.Y[:-1, :]))

print("Verify forecaster valid set...")
print(torch.all(valid_set.forecaster.X[1:, -1, :] == valid_set.forecaster.Y[:-1, :]))

print("Verify forecaster test set...")
print(torch.all(test_set.forecaster.X[1:, -1, :] == test_set.forecaster.Y[:-1, :]))


Verify forecaster train set...
tensor(True)
Verify forecaster valid set...
tensor(True)
Verify forecaster test set...
tensor(True)


In [ ]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
shred.fit(train_set, valid_set, num_epochs=100)

Newest Version
looking good
looking good


In [25]:
prediction = shred(test_set.reconstructor.X).detach().cpu().numpy()
prediction.shape

Newest Version


(300, 6)

In [28]:
prediction

array([[0.44964713, 0.49621034, 0.46844593, 0.41317737, 0.28430694,
        0.24998127],
       [0.43268353, 0.4855593 , 0.45093492, 0.40504658, 0.28789067,
        0.22630996],
       [0.43358487, 0.47830343, 0.44968498, 0.4001704 , 0.2755316 ,
        0.23329918],
       ...,
       [0.454553  , 0.5167656 , 0.47648823, 0.41398188, 0.30312017,
        0.254604  ],
       [0.43435878, 0.4889119 , 0.45707515, 0.4088219 , 0.30662656,
        0.21527702],
       [0.43107188, 0.51079357, 0.45260632, 0.3956104 , 0.30799788,
        0.23773687]], shape=(300, 6), dtype=float32)

In [ ]:
# raw_sensor_measurements = #(ntime, nsensors)

# manager.process()